In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Sioux

In [2]:
numNodes = Compute_Jacobian_Sioux.numNodes;
numLinks = Compute_Jacobian_Sioux.numLinks;
numZones = Compute_Jacobian_Sioux.numZones;
numODpairs = Compute_Jacobian_Sioux.numODpairs;
od_pairs = Compute_Jacobian_Sioux.od_pairs;
link_list_js = Compute_Jacobian_Sioux.link_list_js;
link_length_list = Compute_Jacobian_Sioux.link_length_list;

In [3]:
include("../Julia_files/initia_data.jl");
include("../Julia_files/load_network_uni_class.jl");
include("prepare_data_Sioux.jl");
include("extract_data_Sioux.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

In [4]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [5]:
fcoeffs = [1, 0, 0, 0, .15];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [6]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[0]);

In [7]:
deg = 6
c = 3.5
lam = 1.0

coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[1], arcsDict[0]);

In [8]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], coeffs_dict[(deg, c, lam, 1)]);

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

In [9]:
# weight parameters
gamma1 = 10^4
gamma2 = 1

1

In [10]:
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[0], coeffs_dict[(deg, c, lam, 1)]);

In [11]:
# get arcs data corresponding to initial demands and flows
arcsDict[1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[1]);

In [12]:
linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [13]:
linkCostDicDict[1]["0"], link_length_list[1]

(6.028714025602726,6.0)

In [14]:
jacobiSpiessDict[1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

In [15]:
# maximum number of iterations
N = 50;  

# Armijo rule parameters
rho = 2;
M = 10;

# search direction parameter
epsilon_1 = 0;

# stop criterion parameter
epsilon_2 = 1e-8;

In [16]:
for l = 1:N
    
    jacobDict[l] = jacobiSpiessDict[l]
    
    demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

    descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[0], tapFlowVecDict[l], 
        tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

    searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

    thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

    demandsVecDict[l+1] = similar(demandsVecDict[0]);
    
    demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
        demandsVecDict[0], coeffs_dict[(deg, c, lam, l)], searchDirecDict[l], thetaMaxDict[l], rho, M);
            
    demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);
    
    tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], coeffs_dict[(deg, c, lam, l)]);
        
    arcsDict[l+1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[l+1]);
        
    coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[0]);
    
    objFunTmp = objF(gamma1, gamma2, demandsVecDict[l+1], demandsVecDict[0], 
        coeffs_dict[(deg, c, lam, l+1)])
    
    if objFunTmp > objFunDict[l+1]
        coeffs_dict[(deg, c, lam, l+1)] = coeffs_dict[(deg, c, lam, l)]
    end

    linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], coeffs_dict[(deg, c, lam, l+1)]);
    
    jacobiSpiessDict[l+1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);
        
    demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);
    
    # stopping criterion
    if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
        break
    end
        
end

# normalize objective function value
for l = 1:(length(objFunDict))
    norObjFunDict[l] = objFunDict[l] / objFunDict[1];
end

In [17]:
demandsVecDict

Dict{Any,Any} with 10 entries:
  0 => [100.0,100.0,500.0,200.0,300.0,500.0,800.0,500.0,1300.0,500.0  …  400.0,…
  4 => [99.995,100.05,499.71,200.109,299.762,500.244,800.406,499.678,1300.83,50…
  7 => [100.0,100.007,499.963,200.012,299.972,500.034,800.054,499.957,1300.1,50…
  9 => [100.0,100.006,499.969,200.01,299.976,500.03,800.046,499.963,1300.09,500…
  2 => [99.6663,102.14,486.248,205.003,288.799,511.134,818.699,485.086,1338.77,…
  3 => [99.9371,100.434,497.287,201.003,297.779,502.22,803.719,497.04,1307.69,5…
  5 => [99.9999,100.011,499.94,200.022,299.952,500.054,800.087,499.931,1300.17,…
  8 => [100.0,100.006,499.968,200.01,299.976,500.03,800.047,499.963,1300.09,500…
  6 => [100.0,100.008,499.957,200.015,299.967,500.039,800.062,499.95,1300.12,50…
  1 => [98.7226,108.564,444.608,220.179,255.035,544.795,875.241,440.222,1456.21…

In [18]:
norObjFunDict

Dict{Any,Any} with 9 entries:
  7 => 5.15219e-7
  4 => 2.82861e-5
  9 => 3.75951e-7
  2 => 0.0617633
  3 => 0.00242982
  5 => 1.28861e-6
  8 => 3.79069e-7
  6 => 6.7391e-7
  1 => 1.0

In [19]:
coeffs_dict

Dict{Any,Any} with 9 entries:
  (6,3.5,1.0,1) => [1.0,0.0516668,-0.164568,0.111851,0.137789,-0.0102345,0.0020…
  (6,3.5,1.0,8) => [1.0,0.0088589,-0.0530469,0.0809681,0.103223,0.00940095,-6.7…
  (6,3.5,1.0,2) => [1.0,0.0208211,-0.0938824,0.106842,0.103747,0.0049745,0.0007…
  (6,3.5,1.0,7) => [1.0,0.0088589,-0.0530469,0.0809681,0.103223,0.00940095,-6.7…
  (6,3.5,1.0,4) => [1.0,0.0088699,-0.0530624,0.0809401,0.10328,0.00937078,-6.23…
  (6,3.5,1.0,5) => [1.0,0.0088827,-0.0531391,0.0810842,0.10318,0.00939558,-6.36…
  (6,3.5,1.0,3) => [1.0,0.00874261,-0.0525185,0.0800535,0.103932,0.00915002,-3.…
  (6,3.5,1.0,9) => [1.0,0.0088589,-0.0530469,0.0809681,0.103223,0.00940095,-6.7…
  (6,3.5,1.0,6) => [1.0,0.0088589,-0.0530469,0.0809681,0.103223,0.00940095,-6.7…

In [20]:
outfile = open("./results/norObjFunDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, norObjFunDict)

close(outfile)

In [21]:
outfile = open("./results/demandsDiffDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDiffDict)

close(outfile)

In [22]:
outfile = open("./results/objInvVIDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, objInvVIDict)

close(outfile)

In [23]:
outfile = open("./results/coeffs_dict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)

In [24]:
outfile = open("./results/demandsVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsVecDict)

close(outfile)

In [25]:
outfile = open("./results/demandsDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDict)

close(outfile)

In [26]:
outfile = open("./results/tapFlowDicDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [27]:
outfile = open("./results/tapFlowVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowVecDict)

close(outfile)